In [1]:
from data.Stock import Stock
from data.Options import Options
from data.TS import series_to_supervised

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.models import Model
from keras.layers import LSTM,Dense,Dropout,Input

Using TensorFlow backend.


In [3]:
stock = Stock("AAPL")
optionHandler = Options("AAPL")

2010-01-04 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
RETRIEVING DATA FROM CSV
(1011, 10) (1052, 10)


In [56]:
days = 10

puts,calls = optionHandler.get_puts_calls()
puts = series_to_supervised(puts,days,1)
calls = series_to_supervised(calls,days,1)
target = stock.get_data()['Adj_Close']
target = pd.Series(target).pct_change(days).shift(-days).dropna()#.values
features_futureP = puts.iloc[-days:,:]
features_futureC = calls.iloc[-days:,:]
puts,calls = puts.iloc[:-days,:],calls.iloc[:-days,:]

0 <= -10 Ex: -10, -11.11
1 >-10 and <=-9 Ex: -9.01,-9.99
2 >-9  and <=-8 Ex: -8.01,-8.99
3   -8   -7            
4   -7   -6
5   -6   -5
6   -5   -4
7   -4   -3
8   -3   -2
9   -2   -1 -1.01 9
10  -1    0
11   0    1
12   1    2
13   2    3
14   3    4
15   4    5
16   5    6
17   6    7
18   7    8
19   8    9
20   9    10
21   >=10 
     +11  +10

In [59]:
# target = target * 100
# target = pd.Series(target)

target = target.where(target >= 10,21)
target = target.where(target <= -10,0)
target = target.where((target > -10 and target < 10) , np.ceil(target) + 10)
print(target.head())

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [38]:
from sklearn.model_selection import train_test_split

In [40]:
target_size = target.shape[0] - days
X_trainP, X_testP = train_test_split(puts,test_size=0.2,shuffle=False)
X_trainC, X_testC = train_test_split(calls,test_size=0.2,shuffle=False)
y_train, y_test = train_test_split(target,test_size=0.2,shuffle=False)
print("Puts Train Test {} {} \n Calls Train Test {} {} \n Targets train test {} {}".format(\
                                     X_trainP.shape,
                                     X_testP.shape,
                                     X_trainC.shape,
                                     X_testC.shape,
                                     y_train.shape,
                                     y_test.shape,
                                    ))


Puts Train Test (8, 110) (2, 110) 
 Calls Train Test (8, 110) (2, 110) 
 Targets train test (2101,) (526,)
